# Performing Attacks

In this notebook we will look at performing some client-side attacks on a federated learning system.

We will first make sure install and import the required libraries.

In [1]:
#%pip install -U git+https://github.com/codymlewis/ymir.git git+https://github.com/codymlewis/tenjin.git tqdm

from functools import partial

import tensorflow as tf
import numpy as np
import tenjin
from tqdm.notebook import trange

import ymir

2022-04-20 13:25:58.552989: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-20 13:25:58.553033: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


We will next, write a function for created a LeNET-300-100 model, and set up our datasets for each client.

In [2]:
def create_model(input_shape, output_shape, lr=0.1):
    inputs = tf.keras.layers.Input(shape=input_shape)
    x = tf.keras.layers.Flatten()(inputs)
    x = tf.keras.layers.Dense(300, activation="relu")(x)
    x = tf.keras.layers.Dense(100, activation="relu")(x)
    outputs = tf.keras.layers.Dense(output_shape, activation="softmax")(x)
    model = tf.keras.models.Model(inputs=inputs, outputs=outputs)
    opt = tf.keras.optimizers.SGD(learning_rate=lr)
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
    model.compile(loss=loss_fn, optimizer=opt, metrics=['accuracy'])
    return model

num_clients = 10
dataset = ymir.mp.datasets.Dataset(*tenjin.load('mnist'))
batch_sizes = [32 for _ in range(num_clients)]
data = dataset.fed_split(batch_sizes, ymir.mp.distributions.lda)
train_eval = dataset.get_iter("train", 10_000)
test_eval = dataset.get_iter("test", 10_000)

We will also construct the network and global model as normal.

In [3]:
network = ymir.mp.network.Network()
for d in data:
    network.add_client(ymir.regiment.Scout(create_model(dataset.input_shape, dataset.classes), d, 1, test_data=test_eval))
learner = ymir.garrison.fedavg.Captain(create_model(dataset.input_shape, dataset.classes, lr=1), network)

2022-04-20 13:26:03.005958: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-20 13:26:03.005984: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-20 13:26:03.006000: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (cmla): /proc/driver/nvidia/version does not exist
2022-04-20 13:26:03.006167: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Now, before we start training, we will want go through and convert some of our clients into adversaries. Let's look at a few different
types of attacks.

## Backdoor

We will look at model replacement backdoor attack, we will first setup some variables for the attack, then convert two clients into adversaries.

In [4]:
attack_from, attack_to = 0, 1
num_adversaries = 2

for i in range(num_adversaries):
    ymir.fritz.backdoor.convert(network.clients[i], attack_from, attack_to, np.ones((5, 5, 1)))
    ymir.fritz.scaler.convert(network.clients[i], num_clients - num_adversaries)

We will next create a dataset to evaluate the effectiveness of the attack.

In [5]:
backdoor_eval = dataset.get_iter("test").filter(lambda y: y == attack_from).map(
    partial(ymir.fritz.backdoor.backdoor_map, attack_from, attack_to, np.ones((5, 5, 1)), no_label=True))

Finally, we perform the training loop as normal, we additionally measure attack success rate (ASR) which measures the rate
at which the attack makes the model predict the target class.

In [6]:
for r in (pbar := trange(500)):
    loss = learner.step()
    if r % 10 == 0:
        metrics = learner.model.test_on_batch(*next(test_eval), return_dict=True)
        metrics['ASR'] = np.mean(tf.argmax(learner.model(next(backdoor_eval)[0]), axis=1).numpy() == 1)
        pbar.set_postfix(metrics)

  0%|          | 0/500 [00:00<?, ?it/s]

## Label flipping

The process of label flipping is reasonably similar.

In [7]:
# First reset the network and global model
network = ymir.mp.network.Network()
for d in data:
    network.add_client(ymir.regiment.Scout(create_model(dataset.input_shape, dataset.classes), d, 1, test_data=test_eval))
learner = ymir.garrison.fedavg.Captain(create_model(dataset.input_shape, dataset.classes, lr=1), network)
# Convert the adversaries
for i in range(num_adversaries):
    network.clients[i].data = dataset.get_iter("train", batch_sizes[i])
    ymir.fritz.labelflipper.convert(network.clients[i], attack_from, attack_to)
# Construct a testing dataset
labelflipper_eval = dataset.get_iter("test").filter(lambda y: y == attack_from)
# Perform the learning loop
for r in (pbar := trange(500)):
    loss = learner.step()
    if r % 10 == 0:
        metrics = learner.model.test_on_batch(*next(test_eval), return_dict=True)
        metrics['ASR'] = np.mean(tf.argmax(learner.model(next(labelflipper_eval)[0]), axis=1).numpy() == 1)
        pbar.set_postfix(metrics)

  0%|          | 0/500 [00:00<?, ?it/s]

## Free Riding

We also follow a similar process for the free riding attack.

In [8]:
# First reset the network and global model
network = ymir.mp.network.Network()
for d in data:
    network.add_client(ymir.regiment.Scout(create_model(dataset.input_shape, dataset.classes), d, 1, test_data=test_eval))
learner = ymir.garrison.fedavg.Captain(create_model(dataset.input_shape, dataset.classes, lr=1), network)
# Convert the adversaries
for i in range(num_adversaries):
    ymir.fritz.freerider.convert(network.clients[i], "advanced delta")
# Perform the learning loop
for r in (pbar := trange(500)):
    loss = learner.step()
    if r % 10 == 0:
        metrics = learner.model.test_on_batch(*next(test_eval), return_dict=True)
        pbar.set_postfix(metrics)

  0%|          | 0/500 [00:00<?, ?it/s]